```{code-block}
:class: thebe, thebe-init
# Automatic import for live code
import numpy as np
from shrbk.plot import *
```

 (sec:measuring-bias)=
# Measuring Bias

In this section we will discuss several methods to measure shear bias.


## Shear bias from individual galaxies

{cite}`PKSB17` present a shape-noise-free shear bias estimator from individual simulated galaxies. The principle is similar to the metacalibration technique: A small shear is applied to a galaxy image, and the response matrix of the measured ellipticity to the shear is determined by finite differences.

In general, the shear estimate for a single galaxy is dominated by shape noise. To beat down this noise, traditional methods use a very large number of galaxies. Some additional improvement can be gained by using rotated versions of the same galaxy, such that the sum of the intrinsic ellipticities is zero. The resulting shear estimate has a reduced variance.

However, this does not account for the variance from the measured ellipticity, which does not only depend on the intrinsic one, but on the PSF, pixel noise, etc. In addition, all versions of the rotated galaxy need to be detected and have an measured shape, and therefore, selection biases are difficult to quantify with this method.

To reduce shape noise (to virtually zero), {cite}`PKSB17` uses the same noise realisation is used for each sheared, and the unsheared image. The resulting estimate of the response per galaxy is highly precise. It is still not very accurate, since the measured value depends on the noise realisation. To increase the accuracy, a large number of simulated galaxies with different noise realisations is required. These galaxies are however anyway simulated to cover the large parameter space of galaxy properties.

## Example
Here we compute the individual shear bias from image simulations. This method is implemented in the library `shear_bias`.

In [1]:
# Load the library
import shear_bias as sb

# Debugging
import imp
imp.reload(sb)

<module 'shear_bias' from '/local/home/mkilbing/miniconda3/envs/shear/lib/python3.9/site-packages/shear_bias-0.3-py3.9-linux-x86_64.egg/shear_bias/__init__.py'>

### Set up

#### Shear values

First, set the value of the small shear $\Delta g$ to add to the galaxy images.

In [2]:
dg = 0.02

Define the five steps, one in each positive and negative direction along the two coordinate axis, plus the original unchanged image (0, 0).

In [3]:
g_steps = [(-1, 0), (0, -1), (1, 0), (0, 1), (0, 0)]

Create dictionary of shear values with step tuples as keys.

In [4]:
g_dict = {}
for step in g_steps:
    g_dict[step] = (step[0] * dg, step[1] * dg)
g_values = g_dict.values()

#### Number of images

The number of galaxy postage stamps per image is nxy_tiles^2.
If this number is modified, all output files from a previous
run should be deleted.

In [5]:
nxy_tiles = 4

The number of files with different constant shear and PSF.
This parameter can be changed without deleting previous output files.

Since processing takes quite some time, start with a small value for `nfiles`, and run the rest of this example. This value can be increased later, to download and process more image files without re-doing previously computed results.

In [6]:
nfiles = 2

### Downloading image meta-data

First, download `great3` meta-data and config files to create images.

In [8]:
# `great3` branch
great3_branch = 'control/space/constant'

# Remote and local image directories
remote_cfg_dir = 'https://github.com/martinkilbinger/ede18_shear_cal/blob/master/data/great3/{}'\
                 ''.format(great3_branch)
cfg_dir = 'images/config/great3_csc'   
n_downloaded = sb.download_great3_data(cfg_dir, remote_cfg_dir, great3_branch, nfiles, mode='?raw=true')

Creating galsim/great3 input dir 'images/config/great3_csc'
MKDEBUG
MKDEBUG
MKDEBUG
MKDEBUG
MKDEBUG
MKDEBUG
6 files downloaded
*** End download_great3_data ***


### Create images
Run `galsim` to create galaxy images.

In [10]:
# `great3` config files for galsim 
galsim_config_fname = 'csc_multishear.yaml'
galsim_config_psf_fname = 'csc_psf.yaml'
galsim_config_dir = 'images/config/galsim'
galsim_config_path = '{}/{}'.format(galsim_config_dir, galsim_config_fname)
galsim_config_psf_path = '{}/{}'.format(galsim_config_dir, galsim_config_psf_fname)

images_base_dir = 'images/galsim'

# Image file name formats
output_gal_fname_format = 'image-%03d-%1d.fits'
output_psf_fname_format = 'starfield_image-%03d-%1d.fits'


sb.create_all_sims_great3(g_values, galsim_config_path, galsim_config_psf_path, \
                          cfg_dir, images_base_dir, output_gal_fname_format, \
                          output_psf_fname_format, nxy_tiles=nxy_tiles, nfiles=nfiles)

*** Start create_all_sims_great3 ***
For shear (-0.02,0.0), 2 images need to be created, running 'galsim'
running galsim images/config/galsim/csc_multishear.yaml gal.shear.g1=-0.02 gal.shear.g2=0.0 input.catalog.dir=images/config/great3_csc input.dict.dir=images/config/great3_csc output.dir=images/galsim/g1_-0.02_g2_0.0 output.file_name.format=image-%03d-%1d.fits output.nfiles=2 image.nx_tiles=4 image.ny_tiles=4


FileNotFoundError: [Errno 2] No such file or directory: 'galsim'